In [275]:
import re
import os
import json
import spacy
import random
import pdfplumber
from typing import Optional
from spacy.training import Example

In [276]:
TRAIN_DATA = [
    (
        # Ejemplo 1
        '2353 17ABR2024 LEY  No. "POR  MEDIO  DE  LA  CUAL  SE  RECONOCE  COMO  PATRIMONIO '
        'CULTURAL  INMATERIAL  DE  LA NACION  EL  FESTIVAL  PROVINCIANO  DE  ACORDEONES,  CANCION '
        'INÉDITA  Y  PIQUERIA,  DEL  MUNICIPIO  DE  PIVIJAY  -  MAGDALENA  Y  SE  DICTAN  OTRAS '
        'DISPOSICIONES" El  Congreso  de  Colombia,',
        {"entities": [
            (0, 4, "NUMERO_LEY"),         # "2353"
            (5, 15, "FECHA"),              # "17ABR2024"
            (25, 272, "EPIGRAFE")          # desde la apertura de comillas hasta justo antes de "El Congreso..."
        ]}
    ),
    (
        # Ejemplo 2 (nota: la fecha se compone de "17MAY" y "2024" separados en el texto; en la anotación se unen)
        '2355 2024 LEY  No. 17MAY "POR  MEDIO  DEL  CUAL  SE  RECONOCE  EL  FESTIVAL  DEPARTAMENTAL '
        'DE  BANDAS  DE  CUNDINAMARCA  COMO  MANIFESTACION  DEL  PATRIMONIO  CULTURAL  INMATERIAL '
        'DE  LA  NACION" El  Congreso  de  Colombia,',
        {"entities": [
            (0, 4, "NUMERO_LEY"),         # "2355"
            (5, 9, "FECHA"),              # "2024" puede interpretarse junto a "17MAY" (ver nota abajo)
            (19, 24, "FECHA"),            # "17MAY"
            (25, 196, "EPIGRAFE")         # epígrafe entre comillas
        ]}
    ),
    (
        # Ejemplo 3: La fecha viene separada en tres líneas ("2356", "8", "2024" y luego "MAY")
        '2356 8 2024 LEY  No. MAY "POR  MEDIO  DE  LA  CUAL  SE  ELIMINAN  BENEFICIOS  Y  SUBROGADOS '
        'PENALES  PARA  QUIENES  SEAN  CONDENADOS  O  ESTÉN  CUMPLIENDO  DETENCION  PREVENTIVA '
        'POR  EL  DELITO  DE  FEMINICIDIO" El  Congreso  de  Colombia,',
        {"entities": [
            (0, 4, "NUMERO_LEY"),         # "2356"
            (5, 6, "FECHA"),              # "8" (día)
            (7, 11, "FECHA"),             # "2024" (año)
            (21, 24, "FECHA"),            # "MAY" (mes)
            (25, 211, "EPIGRAFE")
        ]}
    ),
    (
        # Ejemplo 4: Uso de "EL CONGRESO DE LA REPUBLICA DE COLOMBIA" para marcar el fin del epígrafe
        'LEY  No.2361 14JUNZUZ4 POR  MEDIO  DEL  CUAL  SE  OTORGAN  LINEAMIENTOS  PARA  LA '
        'CREACION  DE  LA  POLITICA  PUBLICA  DE  LACTANCIA  MATERNA,  ALIMENTACION  COMPLEMENTARIA,  '
        'Y  LA  PROMOCION  DE  LOS  BANCOS  DE  LECHE  HUMANA  COMO  COMPONENTE  ANATOMICO  '
        'EL  CONGRESO  DE  LA  REPUBLICA  DE  COLOMBIA  DECRETA:',
        {"entities": [
            (8, 12, "NUMERO_LEY"),         # "2361"
            (13, 22, "FECHA"),              # "14JUNZUZ4" (aquí convendría aplicar correcciones manuales o reglas post-procesado)
            (23, 258, "EPIGRAFE")           # Desde "POR MEDIO DE..." hasta antes de "EL CONGRESO..."
        ]}
    ),
    (
        # Ejemplo 5: Problema en la fecha "14JU Te" y epígrafe sin comillas
        '14JU Te LEY  No.  2360 POR  MEDIO  DE  LA  CUAL  SE  MODIFICA  Y  ADICIONA  LA  LEY  1384  DE  2010 '
        'RECONOCIENDO  PARA  LOS  EFECTOS  DE  ESTA  LEY  COMO  SUJETOS  DE  ESPECIAL  PROTECCION  CONSTITUCIONAL  A  LAS  '
        'PERSONAS  CON SOSPECHA  O  QUE  PADECEN  CANCER EL  CONGRESO  DE  LA  REPUBLICA DECRETA:',
        {"entities": [
            (0, 4, "FECHA"),
            (18, 22, "NUMERO_LEY"),         
            (23, 261, "EPIGRAFE")
        ]}
        
        # # Ejemplo 5: Problema en la fecha "14JU Te" y epígrafe sin comillas
        # '8 FEB 2022\n-\nLEY ORGÁNICA No2199\nLEY ORGANICA No2199 FEB 2022 --------~~~~~~~~~--- "POR MEDIO DE LA CUAL SE DESARROLLA EL ARTÍCULO 325 DE LA "POR MEDIO DE LA CUAL SE DESARROLLA EL ARTICULO LA CONSTITUCIÓN POLÍTICA Y SE EXPIDE EL RÉGIMEN ESPECIAL DE LA CONSTITUCION POLITICA Y SE EXPIDE EL RÉGIMEN ESPECIAL DE LA REGIÓN METROPOLITANA BOGOTÁ - CUNDINAMARCA" REGION METROPOLITANA BOGOTA CUNDINAMARCA" EL CONGRESO DE COLOMBIA EL CONGRESO DE COLOMBIA DECRETA: ',
        # {"entities": [
        #     (0, 4, "FECHA"),
        #     (18, 22, "NUMERO_LEY"),         
        #     (23, 261, "EPIGRAFE")
        # ]}
    )
]

In [277]:
def fix_ocr_artifacts(text: str) -> str:
    # 1) Eliminar líneas repetidas
    lines = text.split("\n")
    new_lines = []
    last_line = None
    for line in lines:
        if line == last_line:
            continue
        new_lines.append(line)
        last_line = line
    text = "\n".join(new_lines)

    # 2) Unir caracteres separados por guiones/tildes
    #    "1-7-A-BR-2-0-24" -> "17ABR2024", "2-3-5-1-" -> "2351"
    text = re.sub(r"([A-Z0-9])[\-\~]+([A-Z0-9])", r"\1\2", text, flags=re.IGNORECASE)

    # 3) Quitar espacios redundantes
    text = re.sub(r"\s+", " ", text).strip()
    return text


In [278]:
def remove_duplicate_letters(text: str) -> str:
    """
    Reemplaza secuencias de caracteres repetidos (ej. 'AA', 'MMM') 
    por una sola ocurrencia (ej. 'A', 'M').
    Efecto: 'AAMMPPLLIAI' -> 'AMPLIAI'.
    """
    return re.sub(r'(.)\1+', r'\1', text)

In [279]:
def limit_repetitions_to_two(text: str) -> str:
    """
    Si encuentra 3 o más repeticiones consecutivas (ej. 'AAAA'), 
    las reduce a 2 (-> 'AA').
    """
    return re.sub(r'(.)\1{2,}', r'\1\1', text)


In [280]:
def unify_hyphens_and_tildes(text: str) -> str:
    """
    Elimina guiones (~, -) entre letras o dígitos consecutivos,
    p.ej. '1-7-A-BR-2-0-24' -> '17ABR2024'.
    """
    import re
    return re.sub(r"([A-Z0-9])[\-\~]+([A-Z0-9])", r"\1\2", text, flags=re.IGNORECASE)


In [281]:
def remove_exact_duplicate_lines(text: str) -> str:
    lines = text.split("\n")
    new_lines = []
    last_line = None
    for line in lines:
        if line == last_line:
            continue
        new_lines.append(line)
        last_line = line
    return "\n".join(new_lines)


In [282]:
##############################################################################
# 1. Función para limpiar el texto OCR: fix_ocr_issues
##############################################################################
def fix_ocr_issues(text: str) -> str:
    """
    Aplica correcciones y filtros a texto proveniente de OCR.
    - Elimina líneas basura.
    - Corrige palabras mal reconocidas.
    - Normaliza saltos de línea y espacios.
    """
    
    # 1) Eliminar líneas repetidas exactas
    text = remove_exact_duplicate_lines(text)

    # 2) Unificar guiones/tildes entre letras o dígitos
    text = unify_hyphens_and_tildes(text)

    # 3) Reducir repeticiones de caracteres (opción 1)
    # text = remove_duplicate_letters(text)

    # o Limitar a 2 repeticiones (opción 2)
    text = limit_repetitions_to_two(text)
    
    # Correcciones específicas
    correcciones = {
        "NACiÓN": "NACIÓN",
        "Colr,mbiana": "Colombiana",
        "Perscnal": "Personal",
        "AérEa": "Aérea",
        "No1956": "No 1956",
        "No.1956": "No. 1956",
        "N°1956": "N° 1956",
        "LEYNo": "LEY No",
        "LEYNo.": "LEY No.",
        "4JUN2019": "4 JUN 2019",
        "Dadaen": "Dada en",
    }
    for error, correccion in correcciones.items():
        text = text.replace(error, correccion)
    
    # Llamar a la nueva función de artefactos
    text = fix_ocr_artifacts(text)

    # Eliminar líneas formadas solo por guiones, tildes o espacios
    lines = text.split("\n")
    clean_lines = []
    for line in lines:
        # Ajusta el patrón si tienes más símbolos basura
        if re.match(r"^[\-\~\s]+$", line.strip()):
            continue
        clean_lines.append(line)
    text = "\n".join(clean_lines)

    # Normalizar saltos de línea y espacios
    text = re.sub(r"(?<=\S)\n(?=\S)", " ", text)  # une líneas sin espacio
    text = re.sub(r"\n{2,}", "\n\n", text)        # limita saltos sucesivos
    text = re.sub(r"[ \t]+", " ", text)           # compacta espacios
    text = text.strip()

    # Correcciones finales (ejemplo: separar "No1234")
    text = re.sub(r"(No|N°)(\d+)", r"\1 \2", text, flags=re.IGNORECASE)
    # Espacios entre dígitos y texto (ej. "17ABR2024" -> "17 ABR2024")
    text = re.sub(r"(\d+)([A-Z]{1,3}\d{4})", r"\1 \2", text)

    return text

In [283]:
##############################################################################
# 2. Función para parsear fechas con regex: parse_date_str
##############################################################################
def parse_date_str(text: str) -> str | None:
    patterns = [
        r"(\d{1,2})\s*(?:de\s*)?([A-Z]+)\s*(?:de\s*)?(\d{4})",  # 4 JUN 2019, 4 de JUNIO de 2019
        r"(\d{1,2})\s+([A-Z]{3,})\s+(\d{4})", #4 JUN 2019
    ]
    month_map = {  # Mapeo de nombres de mes a números (en mayúsculas)
        "ENE": "01", "FEB": "02", "MAR": "03", "ABR": "04",
        "MAY": "05", "JUN": "06", "JUL": "07", "AGO": "08",
        "SEP": "09", "OCT": "10", "NOV": "11", "DIC": "12",
        "ENERO": "01", "FEBRERO": "02", "MARZO": "03", "ABRIL": "04",
        "MAYO": "05", "JUNIO": "06", "JULIO": "07", "AGOSTO": "08",
        "SEPTIEMBRE": "09", "OCTUBRE": "10", "NOVIEMBRE": "11", "DICIEMBRE": "12"
    }

    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            try:
                day = match.group(1).zfill(2)
                month = month_map.get(match.group(2).upper())
                year = match.group(3)

                if month:
                    return f"{day}/{month}/{year}"
            except ValueError:
                continue
    return None

In [284]:
##############################################################################
# 3. Función para parsear tipo de ley y número de ley: parse_ley_and_type
##############################################################################
def parse_ley_and_type(text: str):
    type_map = {
        "orgánica": "orgánica",
        "organica": "orgánica",
        "ordinaria": "ordinaria",
        "estatutaria": "estatutaria",
        "estaturaria": "estatutaria",  # Podría haber errores de OCR
    }

    pattern = r"""
        (?:LEY|LEYES)[\s\n]*  # "LEY" o "LEYES", seguido de espacios/saltos
        (?:             # Grupo opcional para el tipo de ley
            (ESTATUTARIA|ORDINARIA|ORG[AÁ]NICA)  # Tipos de ley
            [\s\n]*     # 1 o más espacios/saltos DESPUÉS del tipo
        )?              # El tipo de ley es OPCIONAL
        (?:N[°ºc]?\.?|No\.?)?  # "N", "No", "N°", "Nº", "No.", "N.", "Nc.", opcional
        [\s\n]*         # 0 o más espacios/saltos
        (\d+)          # Captura el número de la ley
        (?:['´`‘’])?   # Caracteres especiales opcionales después del número
    """
    match = re.search(pattern, text, re.IGNORECASE | re.VERBOSE | re.DOTALL)

    if not match:
        return (None, None)  # Si no hay coincidencia, devuelve (None, None)

    raw_type = match.group(1)  # Captura el grupo 1 (tipo de ley, si existe)
    ley_number = match.group(2)  # Captura el grupo 2 (número de ley)

    if raw_type:
        tipo_ley = type_map.get(raw_type.lower()) #Uso del diccionario
    else:
        tipo_ley = "orgánica"  # Si no hay tipo explícito, asume "orgánica"

    return (tipo_ley, ley_number)

In [285]:
##############################################################################
# 4. Validaciones mínimas: looks_like_number_ley, looks_like_date, looks_like_epigrafe
##############################################################################
def looks_like_number_ley(value: str) -> bool:
    """
    Chequeo mínimo: ¿contiene dígitos?
    """
    return bool(re.search(r"\d+", value)) if value else False


def looks_like_date(candidate: str) -> bool:
    """
    Intenta parsear 'candidate' con parse_date_str.
    Retorna True si lo parsea, False si no.
    """
    return parse_date_str(candidate) is not None


def looks_like_epigrafe(text_epigrafe: str) -> bool:
    """
    Verifica que el epígrafe cumpla criterios mínimos:
    - Longitud
    - Contenga la frase "POR MEDIO DE" (u otra)
    """
    if not text_epigrafe:
        return False
    if len(text_epigrafe) < 30:  # ajusta a tu criterio
        return False
    if "POR MEDIO DE" not in text_epigrafe.upper():
        return False
    return True

In [286]:
##############################################################################
# 5. Función para remover la segunda ocurrencia de "POR MEDIO DE LA CUAL"
#    o cualquier frase que se repita en el epígrafe.
##############################################################################
def remove_second_phrase_occurrence(text: Optional[str], phrase: str) -> Optional[str]:
    """
    Elimina la segunda ocurrencia de 'phrase' en 'text'.
    Si 'text' es None o no contiene dos ocurrencias, no hace nada.
    """
    if not text:
        return text

    text_upper = text.upper()
    phrase_upper = phrase.upper()

    first_idx = text_upper.find(phrase_upper)
    if first_idx == -1:
        return text  # No aparece ni una vez

    second_idx = text_upper.find(phrase_upper, first_idx + len(phrase))
    if second_idx == -1:
        return text  # Solo hay una ocurrencia

    # Eliminar la segunda ocurrencia de 'phrase' (solo la frase, no todo el texto siguiente)
    before = text[:second_idx]
    after = text[second_idx + len(phrase):]
    return before + after

In [287]:
##############################################################################
# 6. Fallback para extraer epígrafe (con fecha y sin fecha)
##############################################################################
def extraer_epigrafe_fallback(text: str, final_fecha: Optional[str]) -> Optional[str]:
    """
    Intenta extraer el epígrafe anclado a la fecha (si la hay).
    Si falla, hace fallback a una búsqueda sin fecha.
    """
    if final_fecha:
        pattern_with_date = rf"{re.escape(final_fecha)}.*?(POR\s+MEDIO\s+DE\s+LA\s+CUAL\s+.*?)(?:\.\s*|\n|EL\s+CONGRESO)"
        match = re.search(pattern_with_date, text, re.IGNORECASE | re.DOTALL)
        if match:
            return match.group(1).strip()

    # Si no encuentra anclado a la fecha, buscamos sin fecha
    pattern_no_date = r"(POR\s+MEDIO\s+DE\s+LA\s+CUAL\s+.*?)(?:\.\s*|\n|EL\s+CONGRESO)"
    match = re.search(pattern_no_date, text, re.IGNORECASE | re.DOTALL)
    if match:
        return match.group(1).strip()

    return None


In [288]:
##############################################################################
# 7. Función para extraer metadatos con spaCy (NER): spacy_extract_metadata
##############################################################################
def spacy_extract_metadata(text: str, nlp) -> dict:
    doc = nlp(text)
    
    # Depuración: imprime las entidades detectadas
    print("=== spaCy Entities ===")
    for ent in doc.ents:
        print(f"Texto: {ent.text} | Label: {ent.label_}")

    result = {
        "numero_ley": None,
        "fecha": None,
        "epigrafe": None
    }
    for ent in doc.ents:
        if ent.label_ == "NUMERO_LEY":
            result["numero_ley"] = ent.text
        elif ent.label_ == "FECHA":
            result["fecha"] = ent.text
        elif ent.label_ == "EPIGRAFE":
            result["epigrafe"] = ent.text
    return result

In [289]:
##############################################################################
# 8. Normalizar la fecha de spaCy con parse_date_str
##############################################################################
def normalize_spacy_date(spacy_fecha: Optional[str]) -> Optional[str]:
    """
    Intenta parsear la fecha devuelta por spaCy con parse_date_str.
    Si es válida, retorna la versión dd/mm/yyyy. Si no, None.
    """
    if not spacy_fecha:
        return None
    parsed = parse_date_str(spacy_fecha)
    return parsed  # None si falla, o "08/02/2022" si funciona


In [290]:
##############################################################################
# 9. Función principal híbrida: extract_metadata_layout_hybrid
##############################################################################
def extract_metadata_layout_hybrid(text_lineal: str, nlp) -> dict:
    """
    Combina spaCy + regex para extraer: numero_ley, fecha, tipo_ley, epigrafe.
    Usa fallback en caso de datos inconsistentes.
    """
    # Limpieza
    text_lineal = fix_ocr_issues(text_lineal)

    # 1) Extraer con spaCy
    spacy_result = spacy_extract_metadata(text_lineal, nlp)

    # 2) Regex
    tipo_ley, ley_number = parse_ley_and_type(text_lineal)
    fecha_regex = parse_date_str(text_lineal)

    # 3) Normalizar la fecha de spaCy
    spacy_fecha = spacy_result["fecha"]
    spacy_fecha_normalizada = normalize_spacy_date(spacy_fecha)

    # 4) Decidir la fecha final
    if spacy_fecha_normalizada:
        final_fecha = spacy_fecha_normalizada
    else:
        final_fecha = fecha_regex

    # 5) Epígrafe (anclado a la fecha y fallback sin fecha)
    epigrafe_regex = extraer_epigrafe_fallback(text_lineal, final_fecha)

    # 6) Número de ley (fallback)
    spacy_numero_ley = spacy_result["numero_ley"]
    if not spacy_numero_ley or not looks_like_number_ley(spacy_numero_ley):
        spacy_numero_ley = ley_number

    # 7) Epígrafe (fallback spaCy vs regex)
    spacy_epigrafe = spacy_result["epigrafe"]
    if spacy_epigrafe and looks_like_epigrafe(spacy_epigrafe):
        final_epigrafe = spacy_epigrafe
    else:
        final_epigrafe = epigrafe_regex

    # 8) (Opcional) Eliminar segunda ocurrencia de "POR MEDIO DE LA CUAL"
    final_epigrafe = remove_second_phrase_occurrence(final_epigrafe, "POR MEDIO DE LA CUAL")

    # 9) Armar el dict final
    combined = {
        "numero_ley": spacy_numero_ley,
        "fecha": final_fecha,
        "tipo_ley": tipo_ley,
        "epigrafe": final_epigrafe,
    }
    return combined

In [291]:
##############################################################################
# 10. Función para procesar TODAS las páginas y combinar resultados
##############################################################################
def extract_metadata_from_pdf_all_pages(pdf_path: str, nlp=None) -> dict:
    final_metadata = {
        "numero_ley": None,
        "fecha": None,
        "tipo_ley": None,
        "epigrafe": None
    }

    with pdfplumber.open(pdf_path) as pdf:
        for page_index, page in enumerate(pdf.pages):
            page_text = page.extract_text() or ""
            # Limpieza OCR
            page_text = fix_ocr_issues(page_text)

            # Extraer metadatos de ESTA página
            partial_metadata = extract_metadata_layout_hybrid(page_text, nlp)

            # Combinar resultados
            # Reglas de ejemplo (ajusta a tu gusto)
            # 1) Si no hay numero_ley, tomarlo
            if partial_metadata["numero_ley"] and not final_metadata["numero_ley"]:
                final_metadata["numero_ley"] = partial_metadata["numero_ley"]

            # 2) Para la fecha, si no la tenemos, la tomamos
            if partial_metadata["fecha"] and not final_metadata["fecha"]:
                final_metadata["fecha"] = partial_metadata["fecha"]

            # 3) tipo_ley: sobrescribir si se detecta en alguna página
            if partial_metadata["tipo_ley"]:
                final_metadata["tipo_ley"] = partial_metadata["tipo_ley"]

            # 4) epigrafe: si no existe en final o el nuevo es más largo, lo sobrescribimos
            if partial_metadata["epigrafe"]:
                if not final_metadata["epigrafe"]:
                    final_metadata["epigrafe"] = partial_metadata["epigrafe"]
                else:
                    if len(partial_metadata["epigrafe"]) > len(final_metadata["epigrafe"]):
                        final_metadata["epigrafe"] = partial_metadata["epigrafe"]

            # (Opcional) Parar si ya tenemos todo
            # if all(final_metadata[k] for k in final_metadata):
            #     break

    return final_metadata


In [292]:
##############################################################################
# 11. (Opcional) Entrenar y cargar un modelo spaCy
##############################################################################
def train_spacy_model(train_data, output_dir="modelo_leyes_epigrafe"):
    """
    Entrena un modelo spaCy NER con tus entidades personalizadas 
    (NUMERO_LEY, FECHA, EPIGRAFE, etc.).
    """
    nlp = spacy.blank("es")
    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe("ner", last=True)
    else:
        ner = nlp.get_pipe("ner")

    # Añadir etiquetas
    for text, annotations in train_data:
        for start, end, label in annotations["entities"]:
            ner.add_label(label)

    nlp.initialize()

    examples = []
    for text, annotations in train_data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        examples.append(example)

    for i in range(10):
        random.shuffle(examples)
        losses = {}
        for example in examples:
            nlp.update([example], losses=losses)
        print(f"Época {i}, pérdidas: {losses}")

    nlp.to_disk(output_dir)
    print(f"Modelo guardado en: {output_dir}")
    return nlp


In [293]:
def load_spacy_model(model_path="modelo_leyes_epigrafe"):
    """
    Carga el modelo spaCy entrenado desde disco.
    """
    return spacy.load(model_path)

nlp_trained = load_spacy_model("modelo_leyes_epigrafe")


In [294]:
def procesar_pdfs_en_carpeta(carpeta_pdf, carpeta_salida, nlp):
    """
    Procesa todos los archivos PDF en una carpeta y guarda los metadatos en archivos JSON separados,
    usando la función extract_metadata_from_pdf_all_pages.
    """
    # Crear carpeta de salida si no existe
    os.makedirs(carpeta_salida, exist_ok=True)

    for archivo in os.listdir(carpeta_pdf):
        if archivo.lower().endswith(".pdf"):
            ruta_pdf = os.path.join(carpeta_pdf, archivo)

            # Llama a la función que procesa todas las páginas del PDF
            metadatos = extract_metadata_from_pdf_all_pages(ruta_pdf, nlp)

            # Nombre de salida JSON
            nombre_json = os.path.splitext(archivo)[0] + ".json"
            ruta_json = os.path.join(carpeta_salida, nombre_json)

            # Guardar en JSON
            with open(ruta_json, "w", encoding="utf-8") as f:
                json.dump(metadatos, f, ensure_ascii=False, indent=4)

            print(f"Metadatos guardados en: {ruta_json}")


# --- Ejemplo de Uso --- #
if __name__ == "__main__":
    # Rutas de carpeta
    carpeta_pdf = r"c:/Users/Jorge/OneDrive/Documents/proyect/document/leyes"
    carpeta_salida = r"c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output_2024"

    # Cargar tu modelo spaCy (si ya lo tienes entrenado)
    nlp_trained = load_spacy_model("modelo_leyes_epigrafe")  # ejemplo
    # O si estás usando un EntityRuler, define nlp_trained antes.

    # Llamar a la función
    procesar_pdfs_en_carpeta(carpeta_pdf, carpeta_salida, nlp_trained)

=== spaCy Entities ===
Texto: 8 | Label: FECHA
Texto: FEB | Label: FECHA
Texto: 2022 | Label: NUMERO_LEY
Texto: FEB | Label: FECHA
Texto: 2022 | Label: NUMERO_LEY
Texto: ° | Label: NUMERO_LEY
Texto: ° | Label: FECHA
Texto: oro | Label: FECHA
Texto: pop | Label: FECHA
Texto: itali nta an a BB oo gg oo tatá - CC uu nn dd ini an mam arca ar ,c a, dd ee fif nin ir ir yy rr ee gg lala mm ene tn at ra r ssuu ff uu nn cc ioi no an ma im enie ton , to, eenn eell mm aa rr cc oo dd ee ll aa aa uu tt oo nn oo mm iaí a rr ee cc oo nn oo cc idi ad a aa ss uu ss ii nn tt ee gg rara ntn et se s pp oo rr ll aa CCoonnsstittiutucicoinó n PP oo ll ií tt icic aa . AA RR TT ICÍC ULU OL O 22 °° | Label: EPIGRAFE
Texto: ibi | Label: FECHA
Texto: lb | Label: FECHA
Texto: ií | Label: FECHA
Texto: ut | Label: FECHA
Texto: nu | Label: FECHA
Texto: oó | Label: FECHA
Texto: ici | Label: FECHA
Texto: trt | Label: FECHA
Texto: ° | Label: FECHA
Texto: ° | Label: FECHA
Texto: vati | Label: FECHA
Texto: ir | Label: FEC

In [295]:


# pdf_path = r"C:\Users\Jorge\OneDrive\Documents\proyect\document\leyes\LEY-2351-2024.pdf"

# all_text = ""
# with pdfplumber.open(pdf_path) as pdf:
#     for i, page in enumerate(pdf.pages):
#         page_text = page.extract_text() or ""
#         print(f"--- Página {i} ---")
#         print(repr(page_text))
#         print("---------------")
#         all_text += page_text + "\n"